In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from geopy.distance import geodesic
import math

# 读取数据
df = pd.read_csv("D:/data/raw/bike_orders_20210501_20210503.csv")

# 基础信息
# print(f"字段检查：{df.info()}")
# print(f"描述性统计：\n{df.describe()}")
# print(f"数据形状：\n{df.shape}")

# 缺失值
missing_count = df.isnull().sum()
print(f"缺失值数量：{missing_count}")

# 处理缺失值
df = df.drop('COM_ID', axis=1)

# 重复值
duplicate_count = df.duplicated().sum()
print(f"重复值有：{duplicate_count}")

# 重复值处理
df.drop_duplicates(inplace=True)
print(f"重复值有：{df.duplicated().sum()}")


# 转换时间
df['START_TIME'] = pd.to_datetime(df['START_TIME'], errors='coerce')
df['END_TIME'] = pd.to_datetime(df['END_TIME'], errors='coerce')

# 验证坐标是否在合理范围之内
df = df[(df['START_LAT'] >= 22.4) & (df['START_LAT'] <= 22.9) &
        (df['START_LNG'] >= 113.7) & (df['START_LNG'] <= 114.7)]


# 骑行时间
df['ride_duration'] = (df['END_TIME'] - df['START_TIME']).dt.total_seconds()/60


# 骑行距离
def calculate_distance_haversine(row):
    lat1, lon1 = row['START_LAT'], row['START_LNG']
    lat2, lon2 = row['END_LAT'], row['END_LNG']

    R = 6371 # 地球半径
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = (math.sin(dlat/2) * math.sin(dlat/2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon/2) * math.sin(dlon/2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

df['distance_km'] = df.apply(calculate_distance_haversine, axis=1)


# 业务特征
df['date'] = df['START_TIME'].dt.date
df['hour'] = df['START_TIME'].dt.hour
df['day_of_week'] = df['START_TIME'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['time_period'] = pd.cut(df['hour'],
                           bins = [0, 6, 9, 12, 14, 17, 19, 24],
                           labels = ['深夜', '早高峰', '上午', '午间', '下午', '晚高峰', '夜晚'],
                           right = False)

df['day_of_week_c'] = df['day_of_week'].map({0: '星期一',1: '星期二',2: '星期三',3: "星期四",4: '星期五',5: '星期六',6: '星期天'})

# 骑行速度特征
df['speed_kmh'] = df['distance_km'] / (df['ride_duration'] / 60)

user_stats = df.groupby('USER_ID').agg({
    'ride_duration': ['count', 'mean', 'std'],
    'distance_km': ['mean', 'std']
}).round(2)
user_stats.columns = ['ride_count', 'avg_duration', 'std_duration', 'avg_distance', 'std_distance']
user_stats = user_stats.reset_index()

# 用户分群
def classify_user(row):
    if row['ride_count'] >= 10:
        return '高频用户'
    elif row['ride_count'] >= 5:
        return '中频用户'
    else:
        return '低频用户'

user_stats['user_segment'] = user_stats.apply(classify_user, axis=1)
df = df.merge(user_stats[['USER_ID', 'user_segment']], on='USER_ID', how='left')


df.to_csv("D:/data/raw/bike_orders_cleaned.csv", index = False)
print(f'数据保存成功，共{len(df)}条')